In [67]:
import numpy as np
import pandas as pd

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pyspark.sql import *
from pyspark.sql.functions import col, lower, regexp_replace, split, size, UserDefinedFunction
from pyspark.sql.types import StringType, IntegerType


In [2]:
# df_path = "../csv_data/top-10-sample-output.csv"
df_path = "../csv_data/sample-output.csv"
df = spark.read.csv(df_path, inferSchema=True, header=True, multiLine=True, escape='"')
pdf = pd.read_csv(df_path, index_col=0)

In [12]:
row = df.take(5)[1]

In [23]:
import re

In [64]:
def citation_counter(citation_source):
    def _count_citations(text):
        matches = re.findall(f"{{cite {citation_source} (.*?)}}", text, re.IGNORECASE)
        return len(matches)
    return _count_citations
    

In [69]:
book_citations_count = UserDefinedFunction(citation_counter("book"), IntegerType())
journal_citations_count = UserDefinedFunction(citation_counter("journal"), IntegerType())

In [74]:
data = df.withColumn("book_citations", book_citations_count("text"))\
  .withColumn("journal_citations", journal_citations_count("text"))